In [88]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [89]:
#load stock market data
nyse_df_2019=pd.read_csv('data/NYA.csv')
nyse_df_2019.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-02,11238.769531,11407.799805,11204.280273,11383.530273,11383.530273,3733160000
1,2019-01-03,11343.790039,11343.790039,11169.459961,11190.440430,11190.440430,3822860000
2,2019-01-04,11323.730469,11559.019531,11323.730469,11533.339844,11533.339844,4213410000
3,2019-01-07,11536.049805,11678.969727,11504.769531,11605.959961,11605.959961,4104710000
4,2019-01-08,11702.500000,11727.750000,11621.150391,11716.230469,11716.230469,4081030000


In [3]:
nyse_df_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 7 columns):
Date         251 non-null object
Open         251 non-null float64
High         251 non-null float64
Low          251 non-null float64
Close        251 non-null float64
Adj Close    251 non-null float64
Volume       251 non-null int64
dtypes: float64(5), int64(1), object(1)
memory usage: 13.8+ KB


In [4]:
#Load Weather data
# IV. FORMAT OF "ghcnd-stations.txt"
#------------------------------
#Variable   Columns   Type
#------------------------------
#ID            1-11   Character
#LATITUDE     13-20   Real
#LONGITUDE    22-30   Real
#ELEVATION    32-37   Real
#STATE        39-40   Character
#NAME         42-71   Character
#GSN FLAG     73-75   Character
#HCN/CRN FLAG 77-79   Character
#WMO ID       81-85   Character
#------------------------------

In [95]:
stations_file = open('data/aws-data/ghcnd-stations.txt', 'r') 
lines = stations_file.readlines()

newyork_stations =  []

for line in lines: 
    line = line.strip()
    newyork_stations.append((line[:11], line[12:20], line[21:30],line[38:40]))
    
newyork_stations_df = pd.DataFrame(newyork_stations)
newyork_stations_df = newyork_station_df[newyork_station_df[3] == 'NY']
newyork_stations_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 75594 to 114234
Data columns (total 5 columns):
0          25 non-null object
1          25 non-null object
2          25 non-null object
3          25 non-null object
isInNYC    25 non-null bool
dtypes: bool(1), object(4)
memory usage: 1.0+ KB


In [96]:
newyork_stations_df.head()

,0,1,2,3,isInNYC
75594,US1NYBX0025,40.8599,-73.8457,NY,True
75841,US1NYKN0003,40.6194,-73.9859,NY,True
75842,US1NYKN0025,40.6846,-73.9867,NY,True
75937,US1NYNY0074,40.7969,-73.9330,NY,True
76048,US1NYQN0002,40.7145,-73.8819,NY,True


In [97]:
import geopandas
from shapely.geometry import Point, Polygon

nybb_path = geopandas.datasets.get_path('nybb')

boros = geopandas.read_file(nybb_path)
boros = boros.to_crs(epsg=4326)
boros

,BoroCode,BoroName,Shape_Leng,Shape_Area,geometry
0,5,Staten Island,330470.010332,1.623820e+09,(POLYGON ((-74.05050806403247 40.5664220341607...
1,4,Queens,896344.047763,3.045213e+09,(POLYGON ((-73.83668274106707 40.5949466970158...
2,3,Brooklyn,741080.523166,1.937479e+09,(POLYGON ((-73.86706149472118 40.5820879767934...
3,1,Manhattan,359299.096471,6.364715e+08,(POLYGON ((-74.01092841268031 40.6844914725429...
4,2,Bronx,464392.991824,1.186925e+09,(POLYGON ((-73.89680883223774 40.7958084451597...


In [103]:
def isInNewYorkCity(lon, lat):
    point = Point(float(lat), float(lon))
    for boroPloy in boros['geometry']:
        if boroPloy.contains(point):
            return True
    return False


isNewYorks = []    
for i, row in newyork_stations_df.iterrows():
    isNewYorks.append(isInNewYorkCity(row[1],row[2]))
    
newyork_stations_df['isInNYC']= isNewYorks
newyork_stations_df = newyork_station_df[newyork_stations_df['isInNYC'] == True]
nyc_stations_df = newyork_stations_df.drop([1,2,3,'isInNYC'], axis=1)
nyc_stations_df.head()

,0
75594,US1NYBX0025
75841,US1NYKN0003
75842,US1NYKN0025
75937,US1NYNY0074
76048,US1NYQN0002


In [239]:
def getWeatherInfo(file):
    weather_df = pd.read_csv(file, header=None)
    nyc_weather_df = pd.merge(weather_df, nyc_stations_df, on=[0, 0])
    nyc_weather_df = nyc_weather_df.drop([4,5,6,7],axis=1)
    nyc_weather_df.columns = ['id', 'Date', 'type', 'value']
    nyc_weather_df = nyc_weather_df[(nyc_weather_df['type'] == 'PRCP') | (nyc_weather_df['type'] == 'SNOW') | (nyc_weather_df['type'] == 'SNWD') 
                    | (nyc_weather_df['type'] == 'AWDR') | (nyc_weather_df['type'] == 'AWND')]
    nyc_weather_df = pd.pivot_table(nyc_weather_df, values='value', index=['Date'], columns=['type'], aggfunc=np.mean)
    nyc_weather_df = nyc_weather_df.reset_index()
    nyc_weather_df['Date'] = nyc_weather_df.Date.apply(str)
    nyc_weather_df['Date'] = nyc_weather_df.Date.apply(lambda x: '{}-{}-{}'.format(x[:4], x[4:6], x[6:8]))
    return nyc_weather_df

nyc_weather_df_2019 = getWeatherInfo('data/aws-data/2019.csv')
nyc_weather_df_2019.head()

type,Date,AWND,PRCP,SNOW,SNWD
0,2019-01-01,79.0,233.400000,0.0,0.0
1,2019-01-02,37.5,0.000000,0.0,0.0
2,2019-01-03,49.0,0.333333,0.0,0.0
3,2019-01-04,48.0,0.000000,0.0,0.0
4,2019-01-05,57.0,91.125000,0.0,0.0


In [240]:
def getStockAndWeatherDataFrame(stock_df, weather_df):
    nyc_stocl_weather_df = pd.merge(stock_df, weather_df, on=['Date', 'Date'])
    return nyc_stocl_weather_df

nyc_stock_weather_df_2019 = getStockAndWeatherDataFrame(nyse_df_2019, nyc_weather_df_2019)
nyc_stock_weather_df_2019.head()

,Date,Open,High,Low,Close,Adj Close,Volume,AWND,PRCP,SNOW,SNWD
0,2019-01-02,11238.769531,11407.799805,11204.280273,11383.530273,11383.530273,3733160000,37.5,0.000000,0.0,0.0
1,2019-01-03,11343.790039,11343.790039,11169.459961,11190.440430,11190.440430,3822860000,49.0,0.333333,0.0,0.0
2,2019-01-04,11323.730469,11559.019531,11323.730469,11533.339844,11533.339844,4213410000,48.0,0.000000,0.0,0.0
3,2019-01-07,11536.049805,11678.969727,11504.769531,11605.959961,11605.959961,4104710000,45.5,0.000000,0.0,0.0
4,2019-01-08,11702.500000,11727.750000,11621.150391,11716.230469,11716.230469,4081030000,30.0,29.125000,0.0,0.0


In [241]:
nyc_stock_weather_df_2019.drop(['Adj Close'], inplace=True, axis=1)
nyc_stock_weather_df_2019.head()

,Date,Open,High,Low,Close,Volume,AWND,PRCP,SNOW,SNWD
0,2019-01-02,11238.769531,11407.799805,11204.280273,11383.530273,3733160000,37.5,0.000000,0.0,0.0
1,2019-01-03,11343.790039,11343.790039,11169.459961,11190.440430,3822860000,49.0,0.333333,0.0,0.0
2,2019-01-04,11323.730469,11559.019531,11323.730469,11533.339844,4213410000,48.0,0.000000,0.0,0.0
3,2019-01-07,11536.049805,11678.969727,11504.769531,11605.959961,4104710000,45.5,0.000000,0.0,0.0
4,2019-01-08,11702.500000,11727.750000,11621.150391,11716.230469,4081030000,30.0,29.125000,0.0,0.0


In [242]:
SNOW_HIGH_TH = 25.0
SNOW_MEDIUM_TH = 15.0
nyc_stock_weather_df_2019['SNOW'] = nyc_stock_weather_df_2019.SNOW.apply(lambda x : 0 if x == 0.0 else 1)
nyc_stock_weather_df_2019['SNOW_Intensity'] = nyc_stock_weather_df_2019.SNWD.apply(lambda x: 2 if x > SNOW_HIGH_TH else ( 1 if x > SNOW_MEDIUM_TH else 0))

In [243]:
PRCP_HIGH_TH = 30.0
PRCP_MEDIUM_TH = 20.0
nyc_stock_weather_df_2019['RAIN'] = nyc_stock_weather_df_2019.PRCP.apply(lambda x : 0 if x == 0.0 else 1)
nyc_stock_weather_df_2019['RAIN_Intensity'] = nyc_stock_weather_df_2019.PRCP.apply(lambda x: 2 if x > PRCP_HIGH_TH else ( 1 if x > PRCP_MEDIUM_TH else 0))

In [244]:
AWND_HIGH_TH = 30.0
AWND_MEDIUM_TH = 20.0
nyc_stock_weather_df_2019['WINDY'] = nyc_stock_weather_df_2019.AWND.apply(lambda x: 2 if x > AWND_HIGH_TH else ( 1 if x > AWND_MEDIUM_TH else 0))
nyc_stock_weather_df_2019.drop(['AWND','PRCP','SNWD'], inplace=True, axis=1)
nyc_stock_weather_df_2019.head()

,Date,Open,High,Low,Close,Volume,SNOW,SNOW_Intensity,RAIN,RAIN_Intensity,WINDY
0,2019-01-02,11238.769531,11407.799805,11204.280273,11383.530273,3733160000,0,0,0,0,2
1,2019-01-03,11343.790039,11343.790039,11169.459961,11190.440430,3822860000,0,0,1,0,2
2,2019-01-04,11323.730469,11559.019531,11323.730469,11533.339844,4213410000,0,0,0,0,2
3,2019-01-07,11536.049805,11678.969727,11504.769531,11605.959961,4104710000,0,0,0,0,2
4,2019-01-08,11702.500000,11727.750000,11621.150391,11716.230469,4081030000,0,0,1,1,1


In [249]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [253]:
features = ['Date','Open', 'Low','Close', 'Volume', 'SNOW','SNOW_Intensity','RAIN','RAIN_Intensity','WINDY']

# Separating out the features
x = nyc_stock_weather_df_2019.loc[:, features].values
# Separating out the target
y = nyc_stock_weather_df_2019.loc[:,['Open']].values
# Standardizing the features
x = StandardScaler().fit_transform(x)

ValueError: could not convert string to float: '2019-12-30'

In [12]:
left = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                     'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']})

right = pd.DataFrame({'key1': ['K1', 'K2'],
                       'C': ['C0', 'C2']})

In [13]:
left

,key1,A,B
0,K0,A0,B0
1,K1,A1,B1
2,K1,A2,B2
3,K2,A3,B3


In [14]:
right

,key1,C
0,K1,C0
1,K2,C2


In [15]:
result = pd.merge(left, right, on=['key1', 'key1'])
result

,key1,A,B,C
0,K1,A1,B1,C0
1,K1,A2,B2,C0
2,K2,A3,B3,C2
